In [ ]:
import gradio as gr
import asyncio
import nest_asyncio
from typing import Dict, List
import logging
import json
import numpy as np
import tempfile
import os
import threading
from typing import Optional, Tuple
import io
from openai import OpenAI
# Import necessary LangChain components
from langchain_mcp_tools import convert_mcp_to_langchain_tools
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain.tools import Tool
from langchain_experimental.tools import PythonREPLTool
from langchain_community.tools import ShellTool

import wave
import numpy as np
import scipy.io.wavfile as wavfile


import dotenv
# Load environment variables from .env file
dotenv.load_dotenv()

# Apply nest_asyncio to handle async issues
nest_asyncio.apply()

# Set up logging so we can see what's happening
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# Initialize OpenAI client (using v1+ Python library interface)
client = OpenAI()

class SimpleAudioHandler:
    """Handles speech-to-text and text-to-speech using OpenAI's v1 audio endpoints."""
    def __init__(self):
        self.can_hear = True
        self.can_speak = True

    def listen(self, audio_data: Tuple[int, np.ndarray]) -> str:
        """Transcribe audio_data using OpenAI's Whisper-based endpoint."""
        if not self.can_hear or audio_data is None:
            return ""

        sample_rate, audio_array = audio_data
        # Pack numpy array into WAV bytes
        buf = io.BytesIO()
        buf.name = "audio.wav"  # ensure correct file extension for OpenAI
        with wave.open(buf, 'wb') as wav_file:
            wav_file.setnchannels(1)
            wav_file.setsampwidth(2)
            wav_file.setframerate(sample_rate)
            if audio_array.dtype != np.int16:
                audio_array = (audio_array * 32767).astype(np.int16)
            wav_file.writeframes(audio_array.tobytes())
        buf.seek(0)

        # Call new v1 API
        response = client.audio.transcriptions.create(
            file=buf,
            model="gpt-4o-transcribe"
        )
        return getattr(response, 'text', '')

    def speak(self, text: str) -> Optional[Tuple[int, np.ndarray]]:
        """Generate speech audio from text via OpenAI's TTS endpoint."""
        if not self.can_speak or not text:
            return None

        # Synthesize speech using v1 API
        response = client.audio.speech.create(
            model="gpt-4o-mini-tts",
            input=text,
            voice="ash"
        )
        audio_bytes = response
        if not audio_bytes:
            return None
     # Extract raw bytes
        if hasattr(response, 'content'):
            audio_bytes = response.content  # typical for HTTPX-like response
        elif hasattr(response, 'read'):
            audio_bytes = response.read()   # fallback if streaming
        else:
            # assume it's already bytes
            audio_bytes = response  # type: ignore

        # Ensure bytes-like
        if not isinstance(audio_bytes, (bytes, bytearray)):
            try:
                audio_bytes = bytes(audio_bytes)
            except Exception:
                raise TypeError(f"Could not convert TTS response to bytes: {type(audio_bytes)}")

        # Load WAV bytes into numpy
        tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
        tmp.write(audio_bytes)
        tmp.close()
        # Return filepath for Gradio Audio (type="filepath")
        return tmp.name


class SimpleMultiAgentDashboard:
    """A simplified multi-agent dashboard for students to learn from"""
    
    def __init__(self, mcp_servers: Dict, agent_configs: List[Dict]):
        """Initialize the dashboard with servers and agent configurations"""
        self.mcp_servers = mcp_servers
        self.agent_configs = agent_configs
        self.agents = {}  # Will store our created agents
        self.tools = []   # Will store all available tools
        self.max_iterations = 5  # How many times an agent can use tools
        self.audio = SimpleAudioHandler()  # For voice input/output
        
        # MCP event loop for async operations
        self.mcp_loop = None
        self.mcp_thread = None
        self.cleanup_func = None
        
        # Initialize everything
        self._setup_tools()
        self._create_agents()
        logger.info("✅ Dashboard initialized successfully!")
        
        # Register cleanup on exit
        import atexit
        atexit.register(self._cleanup)
    
    def _cleanup(self):
        """Clean up resources when shutting down"""
        logger.info("Cleaning up...")
        
        # Clean up MCP resources
        if self.cleanup_func and self.mcp_loop:
            try:
                future = asyncio.run_coroutine_threadsafe(
                    self.cleanup_func(),
                    self.mcp_loop
                )
                future.result(timeout=5)
            except Exception as e:
                logger.error(f"Cleanup error: {e}")
        
        # Stop MCP event loop
        if self.mcp_loop:
            self.mcp_loop.call_soon_threadsafe(self.mcp_loop.stop)
            if self.mcp_thread:
                self.mcp_thread.join(timeout=2)
    
    def _setup_tools(self):
        """Set up all the tools our agents can use"""
        
        # 1. Python REPL Tool - for running Python code
        python_tool = PythonREPLTool()
        # Wrap it to handle different input formats
        wrapped_python = Tool(
            name="python_repl",
            description="Execute Python code. Returns the output.",
            func=lambda code: python_tool.run(code if isinstance(code, str) else str(code))
        )
        self.tools.append(wrapped_python)
        
        # 2. Bash/Shell Tool - for running system commands
        bash_tool = Tool(
            name="bash_command",
            description="Execute shell commands.",
            func=ShellTool().run
        )
        self.tools.append(bash_tool)
        
        # 3. MCP Tools - if configured
        if self.mcp_servers:
            self._load_mcp_tools()
        
        logger.info(f"✅ Loaded {len(self.tools)} tools")
    
    def _load_mcp_tools(self):
        """Load MCP (Model Context Protocol) tools"""
        try:
            # Create a dedicated event loop for MCP operations
            self.mcp_loop = asyncio.new_event_loop()
            
            # Run the loop in a separate thread
            self.mcp_thread = threading.Thread(
                target=self.mcp_loop.run_forever,
                daemon=True,
                name="MCP-Thread"
            )
            self.mcp_thread.start()
            logger.info("Started MCP event loop")
            
            # Load MCP tools in the dedicated loop
            future = asyncio.run_coroutine_threadsafe(
                convert_mcp_to_langchain_tools(self.mcp_servers),
                self.mcp_loop
            )
            
            # Get the tools and cleanup function
            mcp_tools, self.cleanup_func = future.result(timeout=30)
            
            # Wrap each MCP tool to work with our system
            for tool in mcp_tools:
                wrapped_tool = self._wrap_mcp_tool(tool)
                self.tools.append(wrapped_tool)
                logger.info(f"  - Added MCP tool: {tool.name}")
            
            logger.info(f"✅ Loaded {len(mcp_tools)} MCP tools")
            
        except Exception as e:
            logger.error(f"❌ Could not load MCP tools: {e}")
            logger.info("Continuing without MCP tools...")
    
    def _wrap_mcp_tool(self, mcp_tool):
        """Wrap an MCP tool to handle async operations properly"""
        def sync_wrapper(input_data):
            """Run the async MCP tool synchronously"""
            try:
                # Handle different input formats
                if isinstance(input_data, dict):
                    final_input = input_data
                else:
                    # Try to create a dict with common parameter names
                    final_input = {"text": str(input_data)}
                
                logger.info(f"Calling MCP tool '{mcp_tool.name}' with: {final_input}")
                
                # Run the async tool in the MCP event loop
                future = asyncio.run_coroutine_threadsafe(
                    mcp_tool.ainvoke(final_input),
                    self.mcp_loop
                )
                
                # Wait for result
                result = future.result(timeout=30)
                
                # Format the result
                if isinstance(result, dict):
                    return json.dumps(result, indent=2)
                else:
                    return str(result)
                    
            except Exception as e:
                logger.error(f"MCP tool error: {e}")
                return f"Error using tool: {str(e)}"
        
        # Create a new Tool with the wrapper
        return Tool(
            name=mcp_tool.name,
            description=mcp_tool.description,
            func=sync_wrapper
        )
    
    def _create_agents(self):
        """Create an agent for each configuration"""
        for config in self.agent_configs:
            agent_name = config["name"]
            
            # Create the language model
            llm = ChatOpenAI(
                model=config.get("model", "gpt-4o"),
                temperature=config.get("temperature", 0.7)
            )
            
            # Select which tools this agent can use
            if agent_name == "Python Coder":
                # Only give code execution tools
                agent_tools = [t for t in self.tools if t.name in ["python_repl", "bash_command"]]
            elif agent_name == "Data Analyst":
                # Give all tools
                agent_tools = self.tools
            else:
                # Give everything except code execution
                agent_tools = [t for t in self.tools if t.name not in ["python_repl", "bash_command"]]
            
            # Create the agent's instructions
            system_message = self._create_system_message(agent_name, agent_tools)
            
            # Create the prompt template
            prompt = ChatPromptTemplate.from_messages([
                ("system", system_message),
                MessagesPlaceholder(variable_name="chat_history"),
                ("human", "{input}"),
                MessagesPlaceholder(variable_name="agent_scratchpad")
            ])
            
            # Create the agent
            agent = create_tool_calling_agent(llm, agent_tools, prompt)
            
            # Create the executor (handles tool execution)
            executor = AgentExecutor(
                agent=agent,
                tools=agent_tools,
                verbose=True,  # Show what the agent is doing
                max_iterations=self.max_iterations
            )
            
            # Store the agent
            self.agents[agent_name] = {
                "executor": executor,
                "tools": agent_tools,
                "config": config
            }
            
            logger.info(f"✅ Created agent: {agent_name}")
    
    def _create_system_message(self, agent_name: str, tools: List) -> str:
        """Create instructions for each agent type"""
        # List all available tools
        tool_list = "\n".join([f"- {tool.name}: {tool.description}" for tool in tools])
        
        if agent_name == "Python Coder":
            return f"""You are a Python programming assistant.

Available tools:
{tool_list}

Instructions:
- Write clean Python code
- Use python_repl to execute code and show results
- Always use print() to display outputs
- Use bash_command for system operations"""
        
        elif agent_name == "Data Analyst":
            return f"""You are a data analysis assistant.

Available tools:
{tool_list}

Instructions:
- Use python_repl for data analysis and visualization
- Always print() results, dataframes, and statistics
- Use appropriate tools for different tasks
- Explain your findings clearly"""
        
        else:
            return f"""You are a helpful assistant.

Available tools:
{tool_list}

Instructions:
- Use tools to help answer questions
- Provide clear and helpful responses
- Explain what you're doing"""
    
    def chat_with_agent(self, message: str, agent_name: str, history: List) -> str:
        """Send a message to an agent and get a response"""
        if agent_name not in self.agents:
            return f"Sorry, I don't know an agent named {agent_name}"
        
        try:
            # Get the agent's executor
            executor = self.agents[agent_name]["executor"]
            
            # Convert history to LangChain format
            chat_history = []
            for msg in history[-10:]:  # Only use last 10 messages
                if msg["role"] == "user":
                    chat_history.append(HumanMessage(content=msg["content"]))
                else:
                    chat_history.append(AIMessage(content=msg["content"]))
            
            # Get the response
            result = executor.invoke({
                "input": message,
                "chat_history": chat_history
            })
            
            # Return the agent's response
            return result["output"]
            
        except Exception as e:
            logger.error(f"Error: {e}")
            return f"Sorry, I encountered an error: {str(e)}"
    
    def process_voice_or_text(self, audio_input, text_input, agent_name, history, speak_response):
        """Process either voice or text input and optionally speak the response"""
        # Get the message from voice or text
        if audio_input is not None:
            message = self.audio.listen(audio_input)
            if not message:
                return None, history, "", "Could not understand audio"
        else:
            message = text_input
        
        if not message.strip():
            return None, history, "", "No message detected"
        
        # Get response from agent
        response = self.chat_with_agent(message, agent_name, history)
        
        # Generate voice response if requested
        voice_response = None
        if speak_response and self.audio.can_speak and len(response) < 1000:
            voice_response = self.audio.speak(response[:500])  # Limit length for TTS
        
        # Update history
        new_history = history + [
            {"role": "user", "content": message},
            {"role": "assistant", "content": response}
        ]
        
        return voice_response, new_history, "", message
    
    def create_gradio_interface(self):
        """Create the Gradio web interface"""
        with gr.Blocks(title="Multi-Agent Assistant", theme=gr.themes.Soft()) as interface:
            # Title
            gr.Markdown("# 🤖 Multi-Agent Voice Assistant with Tools")
            gr.Markdown("Chat with different AI agents using voice or text!")
            
            with gr.Row():
                # Left column - Controls
                with gr.Column(scale=1):
                    # Agent selector
                    agent_dropdown = gr.Dropdown(
                        choices=[config["name"] for config in self.agent_configs],
                        value=self.agent_configs[0]["name"],
                        label="Select Agent"
                    )
                    
                    # Max iterations slider
                    iterations_slider = gr.Slider(
                        minimum=1,
                        maximum=10,
                        value=self.max_iterations,
                        step=1,
                        label="Max Tool Uses",
                        info="How many times the agent can use tools"
                    )
                    
                    # Agent info display
                    agent_info = gr.Markdown("")
                    
                    # Voice input (if available)
                    if self.audio.can_hear:
                        audio_input = gr.Audio(
                            sources=["microphone"],
                            type="numpy",
                            label="🎤 Voice Input (click to record)"
                        )
                    else:
                        audio_input = None
                        gr.Markdown("*Voice input not available - install SpeechRecognition*")
                    
                    # Text input
                    text_input = gr.Textbox(
                        label="Text Input",
                        placeholder="Or type your message here...",
                        lines=3
                    )
                    
                    # Send button
                    send_button = gr.Button("Send", variant="primary")
                    
                    # Voice output toggle (if available)
                    if self.audio.can_speak:
                        speak_toggle = gr.Checkbox(
                            label="🔊 Speak Response",
                            value=True
                        )
                        audio_output = gr.Audio(
                            label="Voice Response",
                            type="numpy",
                            autoplay=True
                        )
                    else:
                        speak_toggle = gr.State(False)
                        audio_output = None
                        gr.Markdown("*Voice output not available - install edge-tts*")
                
                # Right column - Chat
                with gr.Column(scale=2):
                    # Chat history display
                    chatbot = gr.Chatbot(label="Conversation", height=600)
                    
                    # Last input display
                    last_input = gr.Textbox(label="Last Input", interactive=False)
                    
                    # Clear button
                    clear_button = gr.Button("Clear Chat")
            
            # Hidden state to store conversation history
            history_state = gr.State([])
            
            # Function to update agent info
            def update_agent_info(agent_name):
                """Show information about the selected agent"""
                if agent_name in self.agents:
                    tools = self.agents[agent_name]["tools"]
                    tool_names = [tool.name for tool in tools]
                    return f"**{agent_name}**\n\nAvailable tools: {', '.join(tool_names)}"
                return ""
            
            # Function to update max iterations
            def update_iterations(value):
                """Update the max iterations for all agents"""
                self.max_iterations = value
                for agent_data in self.agents.values():
                    agent_data["executor"].max_iterations = value
            
            # Function to clear chat
            def clear_chat():
                """Clear the conversation"""
                return [], [], "", ""
            
            # Connect all the interface elements
            
            # Update agent info when selection changes
            agent_dropdown.change(
                update_agent_info,
                inputs=[agent_dropdown],
                outputs=[agent_info]
            )
            
            # Update iterations when slider changes
            iterations_slider.change(
                update_iterations,
                inputs=[iterations_slider]
            )
            
            # Process message when send button clicked
            send_button.click(
                self.process_voice_or_text,
                inputs=[
                    audio_input if audio_input else gr.State(None),
                    text_input,
                    agent_dropdown,
                    history_state,
                    speak_toggle
                ],
                outputs=[
                    audio_output if audio_output else gr.State(None),
                    history_state,
                    text_input,
                    last_input
                ]
            )
            
            # Update chat display when history changes
            history_state.change(
                lambda h: [(m["content"], None) if m["role"] == "user" 
                          else (None, m["content"]) for m in h],
                inputs=[history_state],
                outputs=[chatbot]
            )
            
            # Clear everything when clear button clicked
            clear_button.click(
                clear_chat,
                outputs=[chatbot, history_state, text_input, last_input]
            )
            
            # Initialize agent info on load
            interface.load(
                lambda: update_agent_info(self.agent_configs[0]["name"]),
                outputs=[agent_info]
            )
        
        return interface


# Example usage
if __name__ == "__main__":
    # Configure MCP servers (optional)
    MCP_SERVERS = {
        "local_http": {"url": "http://127.0.0.1:8000/mcp"}
    }
    
    # Configure agents
    AGENT_CONFIGS = [
        {
            "name": "Python Coder",
            "model": "gpt-4o",
            "temperature": 0.2,
            "description": "Helps with Python programming"
        },
        {
            "name": "Data Analyst", 
            "model": "gpt-4o",
            "temperature": 0.3,
            "description": "Helps with data analysis"
        },
        {
            "name": "Assistant",
            "model": "gpt-4o", 
            "temperature": 0.7,
            "description": "General helpful assistant"
        }
    ]
    
    # Create and launch the dashboard
    dashboard = SimpleMultiAgentDashboard(MCP_SERVERS, AGENT_CONFIGS)
    app = dashboard.create_gradio_interface()
    app.launch(share=True)
    """A simplified multi-agent dashboard for students to learn from"""
    
    def __init__(self, mcp_servers: Dict, agent_configs: List[Dict]):
        """Initialize the dashboard with servers and agent configurations"""
        self.mcp_servers = mcp_servers
        self.agent_configs = agent_configs
        self.agents = {}  # Will store our created agents
        self.tools = []   # Will store all available tools
        self.max_iterations = 5  # How many times an agent can use tools
        
        # Initialize everything
        self._setup_tools()
        self._create_agents()
        logger.info("✅ Dashboard initialized successfully!")
    
    def _setup_tools(self):
        """Set up all the tools our agents can use"""
        
        # 1. Python REPL Tool - for running Python code
        python_tool = PythonREPLTool()
        # Wrap it to handle different input formats
        wrapped_python = Tool(
            name="python_repl",
            description="Execute Python code. Returns the output.",
            func=lambda code: python_tool.run(code if isinstance(code, str) else str(code))
        )
        self.tools.append(wrapped_python)
        
        # 2. Bash/Shell Tool - for running system commands
        bash_tool = Tool(
            name="bash_command",
            description="Execute shell commands.",
            func=ShellTool().run
        )
        self.tools.append(bash_tool)
        
        # 3. MCP Tools - if configured
        if self.mcp_servers:
            self._load_mcp_tools()
        
        logger.info(f"✅ Loaded {len(self.tools)} tools")
    
    def _load_mcp_tools(self):
        """Load MCP (Model Context Protocol) tools"""
        try:
            # Create an event loop for async operations
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)
            
            # Load MCP tools
            mcp_tools, cleanup = loop.run_until_complete(
                convert_mcp_to_langchain_tools(self.mcp_servers)
            )
            
            # Add each MCP tool to our tools list
            for tool in mcp_tools:
                self.tools.append(tool)
                logger.info(f"  - Added MCP tool: {tool.name}")
            
        except Exception as e:
            logger.error(f"❌ Could not load MCP tools: {e}")
            logger.info("Continuing without MCP tools...")
    
    def _create_agents(self):
        """Create an agent for each configuration"""
        for config in self.agent_configs:
            agent_name = config["name"]
            
            # Create the language model
            llm = ChatOpenAI(
                model=config.get("model", "gpt-4o"),
                temperature=config.get("temperature", 0.7)
            )
            
            # Select which tools this agent can use
            if agent_name == "Python Coder":
                # Only give code execution tools
                agent_tools = [t for t in self.tools if t.name in ["python_repl", "bash_command"]]
            elif agent_name == "Data Analyst":
                # Give all tools
                agent_tools = self.tools
            else:
                # Give everything except code execution
                agent_tools = [t for t in self.tools if t.name not in ["python_repl", "bash_command"]]
            
            # Create the agent's instructions
            system_message = self._create_system_message(agent_name, agent_tools)
            
            # Create the prompt template
            prompt = ChatPromptTemplate.from_messages([
                ("system", system_message),
                MessagesPlaceholder(variable_name="chat_history"),
                ("human", "{input}"),
                MessagesPlaceholder(variable_name="agent_scratchpad")
            ])
            
            # Create the agent
            agent = create_tool_calling_agent(llm, agent_tools, prompt)
            
            # Create the executor (handles tool execution)
            executor = AgentExecutor(
                agent=agent,
                tools=agent_tools,
                verbose=True,  # Show what the agent is doing
                max_iterations=self.max_iterations
            )
            
            # Store the agent
            self.agents[agent_name] = {
                "executor": executor,
                "tools": agent_tools,
                "config": config
            }
            
            logger.info(f"✅ Created agent: {agent_name}")
    
    def _create_system_message(self, agent_name: str, tools: List) -> str:
        """Create instructions for each agent type"""
        # List all available tools
        tool_list = "\n".join([f"- {tool.name}: {tool.description}" for tool in tools])
        
        if agent_name == "Python Coder":
            return f"""You are a Python programming assistant.

Available tools:
{tool_list}

Instructions:
- Write clean Python code
- Use python_repl to execute code and show results
- Always use print() to display outputs
- Use bash_command for system operations"""
        
        elif agent_name == "Data Analyst":
            return f"""You are a data analysis assistant.

Available tools:
{tool_list}

Instructions:
- Use python_repl for data analysis and visualization
- Always print() results, dataframes, and statistics
- Use appropriate tools for different tasks
- Explain your findings clearly"""
        
        else:
            return f"""You are a helpful assistant.

Available tools:
{tool_list}

Instructions:
- Use tools to help answer questions
- Provide clear and helpful responses
- Explain what you're doing"""
    
    def chat_with_agent(self, message: str, agent_name: str, history: List) -> str:
        """Send a message to an agent and get a response"""
        if agent_name not in self.agents:
            return f"Sorry, I don't know an agent named {agent_name}"
        
        try:
            # Get the agent's executor
            executor = self.agents[agent_name]["executor"]
            
            # Convert history to LangChain format
            chat_history = []
            for msg in history[-10:]:  # Only use last 10 messages
                if msg["role"] == "user":
                    chat_history.append(HumanMessage(content=msg["content"]))
                else:
                    chat_history.append(AIMessage(content=msg["content"]))
            
            # Get the response
            result = executor.invoke({
                "input": message,
                "chat_history": chat_history
            })
            
            # Return the agent's response
            return result["output"]
            
        except Exception as e:
            logger.error(f"Error: {e}")
            return f"Sorry, I encountered an error: {str(e)}"
    
    def create_gradio_interface(self):
        """Create the Gradio web interface"""
        with gr.Blocks(title="Multi-Agent Assistant", theme=gr.themes.Soft()) as interface:
            # Title
            gr.Markdown("# 🤖 Multi-Agent Assistant with Tools")
            gr.Markdown("Chat with different AI agents that can use various tools!")
            
            with gr.Row():
                # Left column - Controls
                with gr.Column(scale=1):
                    # Agent selector
                    agent_dropdown = gr.Dropdown(
                        choices=[config["name"] for config in self.agent_configs],
                        value=self.agent_configs[0]["name"],
                        label="Select Agent"
                    )
                    
                    # Max iterations slider
                    iterations_slider = gr.Slider(
                        minimum=1,
                        maximum=10,
                        value=self.max_iterations,
                        step=1,
                        label="Max Tool Uses",
                        info="How many times the agent can use tools"
                    )
                    
                    # Agent info display
                    agent_info = gr.Markdown("")
                    
                    # Text input
                    text_input = gr.Textbox(
                        label="Your Message",
                        placeholder="Type your message here...",
                        lines=3
                    )
                    
                    # Send button
                    send_button = gr.Button("Send", variant="primary")
                
                # Right column - Chat
                with gr.Column(scale=2):
                    # Chat history display
                    chatbot = gr.Chatbot(label="Conversation", height=600)
                    
                    # Last input display
                    last_input = gr.Textbox(label="Last Input", interactive=False)
                    
                    # Clear button
                    clear_button = gr.Button("Clear Chat")
            
            # Hidden state to store conversation history
            history_state = gr.State([])
            
            # Function to update agent info
            def update_agent_info(agent_name):
                """Show information about the selected agent"""
                if agent_name in self.agents:
                    tools = self.agents[agent_name]["tools"]
                    tool_names = [tool.name for tool in tools]
                    return f"**{agent_name}**\n\nAvailable tools: {', '.join(tool_names)}"
                return ""
            
            # Function to update max iterations
            def update_iterations(value):
                """Update the max iterations for all agents"""
                self.max_iterations = value
                for agent_data in self.agents.values():
                    agent_data["executor"].max_iterations = value
            
            # Function to process messages
            def process_message(text, agent_name, history):
                """Process a user message and get agent response"""
                if not text.strip():
                    return history, "", "Please enter a message"
                
                # Get agent response
                response = self.chat_with_agent(text, agent_name, history)
                
                # Update history
                new_history = history + [
                    {"role": "user", "content": text},
                    {"role": "assistant", "content": response}
                ]
                
                # Update chat display
                chat_display = []
                for msg in new_history:
                    if msg["role"] == "user":
                        chat_display.append((msg["content"], None))
                    else:
                        chat_display.append((None, msg["content"]))
                
                return new_history, "", text
            
            # Function to clear chat
            def clear_chat():
                """Clear the conversation"""
                return [], [], "", ""
            
            # Connect all the interface elements
            
            # Update agent info when selection changes
            agent_dropdown.change(
                update_agent_info,
                inputs=[agent_dropdown],
                outputs=[agent_info]
            )
            
            # Update iterations when slider changes
            iterations_slider.change(
                update_iterations,
                inputs=[iterations_slider]
            )
            
            # Process message when send button clicked
            send_button.click(
                process_message,
                inputs=[text_input, agent_dropdown, history_state],
                outputs=[history_state, text_input, last_input]
            )
            
            # Update chat display when history changes
            history_state.change(
                lambda h: [(m["content"], None) if m["role"] == "user" 
                          else (None, m["content"]) for m in h],
                inputs=[history_state],
                outputs=[chatbot]
            )
            
            # Clear everything when clear button clicked
            clear_button.click(
                clear_chat,
                outputs=[chatbot, history_state, text_input, last_input]
            )
            
            # Initialize agent info on load
            interface.load(
                lambda: update_agent_info(self.agent_configs[0]["name"]),
                outputs=[agent_info]
            )
        
        return interface


# Example usage
if __name__ == "__main__":
    # Configure MCP servers (optional)
    MCP_SERVERS = {
        "example_server": {"url": "http://localhost:8000/mcp"}
    }
    
    # Configure agents
    AGENT_CONFIGS = [
        {
            "name": "Python Coder",
            "model": "gpt-4o",
            "temperature": 0.2,
            "description": "Helps with Python programming"
        },
        {
            "name": "Data Analyst", 
            "model": "gpt-4o",
            "temperature": 0.3,
            "description": "Helps with data analysis"
        },
        {
            "name": "Assistant",
            "model": "gpt-4o", 
            "temperature": 0.7,
            "description": "General helpful assistant"
        }
    ]
    
    # Create and launch the dashboard
    dashboard = SimpleMultiAgentDashboard(MCP_SERVERS, AGENT_CONFIGS)
    app = dashboard.create_gradio_interface()
    app.launch(share=True)

INFO:__main__:Started MCP event loop
INFO:langchain_mcp_tools.langchain_mcp_tools:MCP server "local_http": initializing with: {'url': 'http://127.0.0.1:8000/mcp'}
INFO:langchain_mcp_tools.langchain_mcp_tools:MCP server "local_http": Pre-validating authentication
ERROR:langchain_mcp_tools.langchain_mcp_tools:MCP server "local_http": error during initialization: MCP server "local_http": Connection failed: All connection attempts failed
ERROR:__main__:❌ Could not load MCP tools: MCP server "local_http": Connection failed: All connection attempts failed
INFO:__main__:Continuing without MCP tools...
INFO:__main__:✅ Loaded 2 tools
INFO:__main__:✅ Created agent: Python Coder
INFO:__main__:✅ Created agent: Data Analyst
INFO:__main__:✅ Created agent: Assistant
INFO:__main__:✅ Dashboard initialized successfully!
/tmp/ipykernel_28259/3952706411.py:485: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecat

* Running on local URL:  http://127.0.0.1:7861


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.gradio.app/v3/tunnel-request "HTTP/1.1 200 OK"


* Running on public URL: https://bc56fd75fbc45c603a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


INFO:httpx:HTTP Request: HEAD https://bc56fd75fbc45c603a.gradio.live "HTTP/1.1 200 OK"


...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/transcriptions "HTTP/1.1 200 OK"




> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


I'm just a program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?

> Finished chain.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/audio/speech "HTTP/1.1 200 OK"
